# Plot linear regression fits

In [ ]:
import numpy as np
import brainlit
import scipy
from brainlit.utils import swc
from cloudvolume.exceptions import SkeletonDecodeError
from brainlit.algorithms.trace_analysis.fit_spline import GeometricGraph
from brainlit.algorithms.trace_analysis.spline_fxns import curvature, torsion
import os
from pathlib import Path
import pandas as pd
from networkx.readwrite import json_graph
import json
import matplotlib.pyplot as plt

## Define brain, find trace data folder

In [ ]:
#specify brain1 or brain2 below
brain = "brain1"
#specify length definition below (0-length from root, 1-relative length from root, 2-relative length along segment)
length_def = 1

spacing = 14

root_dir = Path(os.path.abspath('')).parents[1]
experiment_dir = os.path.join(root_dir, "axon_geometry")
data_dir = os.path.join(experiment_dir, "data", brain)
segments_swc_dir = os.path.join(data_dir, "segments_swc")
trace_data_dir = os.path.join(data_dir, "trace_data")
trace_data_dir = os.path.join(trace_data_dir, str(spacing))
print(f"Directory where swcs reside: {segments_swc_dir}")

## Read trace data

In [ ]:
max_id = 300
seg_lengths = []
mean_torsions = []
mean_curvatures = []
d_from_root = []
torsions = []
curvatures = []
total=0
for i in np.arange(0, max_id):
    i = int(i)
    trace_data_path = os.path.join(trace_data_dir, "{}.npy".format(i))
    if os.path.exists(trace_data_path) is True:
        trace_data = np.load(trace_data_path, allow_pickle=True)
        print("Loaded segment {}".format(i))
        total = total+1

        for node in trace_data:
            # print(node)
            seg_lengths.append(node["seg_length"])
            mean_curvatures.append(node["mean_curvature"])
            mean_torsions.append(node["mean_torsion"])
            _curvatures = node["curvature"]
            _torsions = node["torsion"]
            assert len(_curvatures) == len(_torsions)
            m = len(_curvatures)
            if length_def == 0: # length from root
                d = (node["starting_length"] + np.arange(0, m)*spacing)
            elif length_def == 1: # relative length from root
                d = (node["starting_length"] + np.arange(0, m)*spacing)/(node["seg_length"]+node["starting_length"])
            elif length_def == 2: # relative length along segment
                d = np.arange(0, m)*spacing/node["seg_length"]
            d_from_root.append(d)
            torsions.append(_torsions)
            curvatures.append(_curvatures)
print(f"{total} total axon traces")          
seg_lengths = np.array(seg_lengths)
mean_curvatures = np.array(mean_curvatures)
mean_torsions = np.array(mean_torsions)
d_from_root = np.concatenate(d_from_root)
curvatures = np.concatenate(curvatures)
torsions = np.concatenate(torsions)
# remove all root points
masked_d_from_root = np.ma.masked_less(d_from_root, 1e-16)
d_from_root = masked_d_from_root.compressed()
assert len(np.where(d_from_root < 1e-16)[0]) == 0
curvatures = curvatures[~masked_d_from_root.mask]
torsions = torsions[~masked_d_from_root.mask]

## Fit curvatures

In [ ]:
nonzero_curvatures_masked = np.ma.masked_less(curvatures, 1e-16)
nonzero_curvatures = nonzero_curvatures_masked.compressed()
nonzero_curvatures_d_from_root = d_from_root[nonzero_curvatures_masked.mask == 0]

log_curvatures = np.log10(nonzero_curvatures)
log_curvatures_d_from_root = np.log10(nonzero_curvatures_d_from_root)

log_slope_curvatures, log_intercept_curvatures, _, _, _ = scipy.stats.linregress(
    log_curvatures_d_from_root, log_curvatures
)
log_slope_curvatures = np.around(log_slope_curvatures, decimals=2)
log_intercept_curvatures = np.around(log_intercept_curvatures, decimals=2)
log_curvatures_fit = (
    log_slope_curvatures * log_curvatures_d_from_root + log_intercept_curvatures
)
curvatures_pearson_r, curvatures_p_value = scipy.stats.pearsonr(
    log_curvatures, log_curvatures_fit
)
print(
    log_slope_curvatures,
    log_intercept_curvatures,
    curvatures_pearson_r ** 2,
    curvatures_p_value,
)

n = len(log_curvatures)
print(f"{n} data points in the curvature plot")

## Fit torsions

In [ ]:
nonzero_torsions_masked = np.ma.masked_less(torsions, 1e-16)
nonzero_torsions = nonzero_torsions_masked.compressed()
nonzero_torsions_d_from_root = d_from_root[nonzero_torsions_masked.mask == 0]

log_torsions = np.log10(nonzero_torsions)
log_torsions_d_from_root = np.log10(nonzero_torsions_d_from_root)

log_slope_torsions, log_intercept_torsions, _, _, _ = scipy.stats.linregress(
    log_torsions_d_from_root, log_torsions
)
log_slope_torsions = np.around(log_slope_torsions, decimals=2)
log_intercept_torsions = np.around(log_intercept_torsions, decimals=2)
log_torsions_fit = (
    log_slope_torsions * log_torsions_d_from_root + log_intercept_torsions
)
torsions_pearson_r, torsions_p_value = scipy.stats.pearsonr(
    log_torsions, log_torsions_fit
)
print(
    log_slope_torsions,
    log_intercept_torsions,
    torsions_pearson_r ** 2,
    torsions_p_value,
)

n = len(log_torsions)
print(f"{n} data points in the torsion plot")

## Plot figure

In [ ]:
fig = plt.figure(figsize=(22, 8))
axes = fig.subplots(1, 2)
GRAY = "#999999"
TITLE_TYPE_SETTINGS = {"fontname": "Arial", "size": 20}
SUP_TITLE_TYPE_SETTINGS = {"fontname": "Arial", "size": 24}
plt.rc("font", family="Arial", size=20)

ax = axes[0]
ax.spines["bottom"].set_color(GRAY)
ax.spines["top"].set_color(GRAY)
ax.spines["right"].set_color(GRAY)
ax.spines["left"].set_color(GRAY)
ax.tick_params(axis="both", colors=GRAY, labelsize="large")

if length_def == 0: # length from root
    xlabel = r"$\log$ distance from cell body ($\mu m$)"
elif length_def == 1: # relative length from root
    xlabel = r"$\log$ relative distance from cell body"
elif length_def == 2: # relative length along segment
    xlabel = r"$\log$ relative distance along segment"

ax.scatter(
    log_curvatures_d_from_root,
    log_curvatures,
    marker=".",
    label="Segment",
    color="#377eb8",
)
ax.plot(
    log_curvatures_d_from_root,
    log_curvatures_fit,
    color="#e41a1c",
    lw=2,
    label=r"$y={}x {}{}$".format(
        log_slope_curvatures,
        "+" if np.sign(log_intercept_curvatures) >= 0 else "-",
        np.abs(log_intercept_curvatures),
    ),
)
ax.set_title("Curvature")
ax.set_xlabel(xlabel, fontsize=22)
ax.set_ylabel(r"$\log$ curvature", fontsize=22)
leg = ax.legend(loc=4)
leg.get_frame().set_edgecolor(GRAY)

ax = axes[1]
ax.spines["bottom"].set_color(GRAY)
ax.spines["top"].set_color(GRAY)
ax.spines["right"].set_color(GRAY)
ax.spines["left"].set_color(GRAY)
ax.tick_params(axis="both", colors=GRAY, labelsize="large")

ax.scatter(
    log_torsions_d_from_root,
    log_torsions,
    marker=".",
    label="Segment",
    color="#377eb8",
)
ax.plot(
    log_torsions_d_from_root,
    log_torsions_fit,
    color="#e41a1c",
    lw=2,
    label=r"$y={}x {}{}$".format(
        log_slope_torsions,
        "+" if np.sign(log_intercept_torsions) >= 0 else "-",
        np.abs(log_intercept_torsions),
    ),
)
ax.set_title("Torsion")
ax.set_xlabel(xlabel, fontsize=22)
ax.set_ylabel(r"$\log$ absolute torsion", fontsize=22)
leg = ax.legend(loc=4)
leg.get_frame().set_edgecolor(GRAY)

title = "Brain 1" if brain == "brain1" else "Brain 2"
fig.suptitle(title)

plt.savefig(os.path.join(experiment_dir, "figures/{}_linear_regression_from_root_{}.eps".format(brain, length_def)))
plt.savefig(os.path.join(experiment_dir, "figures/{}_linear_regression_from_root_{}.jpg".format(brain, length_def)))